In [1]:
!gdown --remaining-ok 1TM_VQzVYkWXC0ApXUcsyFYpOJQPIrxqe

Downloading...
From: https://drive.google.com/uc?id=1TM_VQzVYkWXC0ApXUcsyFYpOJQPIrxqe
To: /content/yolo8-20epoch.pt
100% 6.25M/6.25M [00:00<00:00, 25.2MB/s]


In [2]:
!pip install python-chess
!pip install ultralytics
!pip install roboflow

from google.colab import drive
drive.mount('/content/drive')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 41.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=5e45882027d314aa69de23572e02403e6ae9d78d6476d10d398fdbaf51a4e110
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import everything

In [3]:
import chess
import chess.svg
import chess.pgn
from IPython.display import SVG, display
import ultralytics
from ultralytics import YOLO
import os
from PIL import Image
import cv2
from IPython.display import Video
import glob
import matplotlib.pyplot as plt
from roboflow import Roboflow
import warnings
import numpy as np
import random
import re

warnings.filterwarnings('ignore')
ultralytics.checks()


Ultralytics 8.3.112 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.2/107.7 GB disk)


# LOAD MODEL

In [4]:
# chess_model = YOLO("/content/drive/MyDrive/weights/chess/chessPieceV3.pt")
corner_det = YOLO("/content/drive/MyDrive/weights/chess/corner_det.pt")
chess_model = YOLO("/content/yolo8-20epoch.pt")
# print(chess_model1.names)
# print(chess_model2.names)

## LOAD Dataset


In [5]:
# rf = Roboflow(api_key="ozCcRqAwKW37LPNL89gN")
# project = rf.workspace("digimg").project("chess-piece-lwscg")
# version = project.version(1)
# dataset = version.download("yolov11")


# rf = Roboflow(api_key="ozCcRqAwKW37LPNL89gN")
# project = rf.workspace("digimg").project("chess-piece-lwscg")
# version = project.version(2)
# dataset = version.download("yolov11")


# train_paths = [os.path.join("/content/chess-piece--2/train/images", f) for f in os.listdir("/content/chess-piece--2/train/images")]
# valid_paths = [os.path.join("/content/chess-piece--2/valid/images", f) for f in os.listdir("/content/chess-piece--2/valid/images")]


def get_random_path(paths):
    return random.choice(paths)

# Train model

In [6]:
chess_model.names
# chess_model.train(data="/content/chess-piece--2/data.yaml", epochs=50)

{0: 'black-bishop',
 1: 'black-king',
 2: 'black-knight',
 3: 'black-pawn',
 4: 'black-queen',
 5: 'black-rook',
 6: 'white-bishop',
 7: 'white-king',
 8: 'white-knight',
 9: 'white-pawn',
 10: 'white-queen',
 11: 'white-rook'}

# FIND BOARD CORNER AND REMOVE DUPLICATE CORNER


In [7]:

def find_corners(img_path, plot=False):

    img = cv2.imread(img_path)
    detect_res = corner_det.predict(img_path)
    if plot:
        for box in detect_res[0].boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)  # Convert coordinates to integers
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw rectangle
        # Display image
        # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        # plt.show()

    return detect_res

def get_corners(detection_result):

    corners = []
    for box in detection_result[0].boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)  # Convert coordinates to integers
        corners.append(((x1, y1), (x2, y2)))
    return corners

def remove_close_corners(corners, threshold=100, print_res=False):

    unique_corners = []
    if print_res:
        print("before remove =", len(corners))
    for corner in corners:
        if not is_corner_in_list(corner, unique_corners, threshold):
            unique_corners.append(corner)
    if print_res:
        print("after remove =", len(unique_corners))
    return unique_corners

def change_to_points(corners):

    points = []
    for corner in corners:
        centerX = (corner[0][0] + corner[1][0]) / 2
        centerY = (corner[0][1] + corner[1][1]) / 2
        points.append((centerX, centerY))
    return np.array(points, dtype="float32")

def is_corner_in_list(corner, unique_corners, threshold=100):

    for t in unique_corners:
        if is_corners_close(corner, t, threshold):
            return True
    return False

def is_corners_close(corner1, corner2, threshold=100):

    center1 = ((corner1[0][0] + corner1[1][0]) / 2, (corner1[0][1] + corner1[1][1]) / 2)
    center2 = ((corner2[0][0] + corner2[1][0]) / 2, (corner2[0][1] + corner2[1][1]) / 2)
    return abs(center1[0] - center2[0]) < threshold and abs(center1[1] - center2[1]) < threshold

def order_points(points):

    points = np.array(points)

    x_sorted = points[np.argsort(points[:, 0]), :]

    left_pts = x_sorted[:2]
    right_pts = x_sorted[2:]

    top_left = left_pts[np.argmin(left_pts[:, 1])]
    bottom_left = left_pts[np.argmax(left_pts[:, 1])]
    top_right = right_pts[np.argmin(right_pts[:, 1])]
    bottom_right = right_pts[np.argmax(right_pts[:, 1])]

    return np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")

def perspective_transform(img_path, corner_detector=corner_det, output_size=(640, 640), plot=False, expand=0):

    img = cv2.imread(img_path)

    src_points = get_src_points(img_path, corner_detector)
    dst_points = np.array([[0, 0], [output_size[0], 0], [output_size[0], output_size[1]], [0, output_size[1]]], dtype="float32")

    if expand == 0:
        matrix = cv2.getPerspectiveTransform(src_points, dst_points)
        warped = cv2.warpPerspective(img, matrix, output_size)

        # if plot:
        #     plt.imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
        #     plt.show()
        return warped



def get_src_points(img_path, corner_detector=corner_det):
    return np.array(order_points(change_to_points(remove_close_corners(get_corners(find_corners(img_path, corner_detector))))), dtype="float32")

def get_transformed_positions_and_classes(img_path, dst_points = np.array([[0, 0], [640, 0], [640, 640], [0, 640]], dtype="float32"), board_pos=True):
    pos_and_cls = get_positions_and_classes(img_path)
    src_points = get_src_points(img_path)
    ret_list = []
    positions = []
    for dct in pos_and_cls:
        positions.append(dct["position"])
    # Convert the points to the required shape (N, 1, 2)
    points_array = np.array(positions, dtype="float32").reshape(-1, 1, 2)

    # Calculate the perspective transformation matrix
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Apply the transformation
    transformed_points = cv2.perspectiveTransform(points_array, matrix).reshape(-1, 2)
    if board_pos:
        for idx,p in enumerate(transformed_points):
            p = p.tolist()
            pos = get_board_pos(p[0], p[1])
            ret_list.append({"position":pos, "class_name": pos_and_cls[idx]["class_name"]})
        return ret_list
    else:
        for idx,p in enumerate(transformed_points):
            ret_list.append({"position":p.tolist(), "class_name": pos_and_cls[idx]["class_name"]})
        return ret_list

# tmp_point = get_src_points(get_random_path(train_paths), corner_det)
# get_transformed_positions([[0, 0], [100, 100]], tmp_point)

In [8]:

# # Load the image
# img_path = "/content/chess-piece--1/train/images/1595777dfa66e954ae23655743e24809_jpg.rf.67bcd5b5d5605b3c92ab388c80e57b62.jpg"
# img = cv2.imread(img_path)

# # Define the point to plot
# point = (200, 100)  # (x, y)

# # Draw a red dot on the image
# img_with_dot = img.copy()  # Create a copy to avoid altering the original
# cv2.circle(img_with_dot, point, radius=5, color=(0, 0, 255), thickness=-1)  # Red dot

# # Display the image with the red dot
# plt.imshow(cv2.cvtColor(img_with_dot, cv2.COLOR_BGR2RGB))
# plt.title("Image with Red Dot")
# plt.show()

# new_point = get_transformed_positions(point, get_src_points(img_path, corner_det))[0][0]
# new_point = (int(new_point[0]), int(new_point[1]))
# print(new_point)
# warped = perspective_transform(img_path, corner_det, output_size=(640, 640), plot=False)
# warped_img_with_dot = warped.copy()  # Create a copy to avoid altering the original
# cv2.circle(warped_img_with_dot, new_point, radius=5, color=(0, 0, 255), thickness=-1)  # Red dot

# # Display the image with the red dot
# plt.imshow(cv2.cvtColor(warped_img_with_dot, cv2.COLOR_BGR2RGB))
# plt.title("Image with Red Dot")
# plt.show()

In [9]:
# img_path = "/content/chess-piece--2/train/images/4d6b667ecbd41ebd603b38848366d9d0_jpg.rf.ce552028199c409d44fcbd51d96ec3a7.jpg"
# get_detect_res(img_path, plot=True)
# find_corners(img_path, corner_det, plot=True)
# mark_red_dots(img_path)

In [10]:


def get_detect_res(img_path, model=chess_model, plot=False):
    result = model.predict(img_path)
    if plot:
        img = result[0].plot()
        plt.imshow(img)
        plt.axis("off")
        plt.show()
    return result


def get_positions_and_classes(img_path, method="default"):
    """
    Extract positions and their associated classes from detection results.

    Args:
        img_path (str): Path to the image.
        method (str): Method to calculate positions ("default", "centerX", "centerY", "centerXY").

    Returns:
        list: A list of dictionaries containing positions and class names.
    """
    pos_and_cls = []
    result = get_detect_res(img_path)

    # Extract bounding boxes and classes
    boxes = result[0].boxes.xyxy.cpu().numpy()  # Bounding box coordinates
    classes = result[0].boxes.cls.cpu().numpy()  # Class indices
    names = result[0].names  # Class name mapping

    for i, (x1, y1, x2, y2) in enumerate(boxes):
        dy = abs(y2 - y1)
        dx = abs(x2 - x1)
        if method == "default":
            if dx <= dy:
                method = "centerX"
            else:
                method = "centerXY"

        # Calculate position based on the method
        if method == "centerXY":
            position = (int((x1 + x2) / 2), int((y1 + y2) / 2))
        elif method == "centerX":
            position = (int((x1 + x2) / 2), int(y2))
        elif method == "centerY":
            position = (int(x2), int((y1 + y2) / 2))


        # Get the class name from the index
        class_id = int(classes[i])
        class_name = names[class_id]

        # Add position and class to the result
        pos_and_cls.append({"position": position, "class_name": class_name})

    return pos_and_cls

def mark_red_dots(img_path):
    img = perspective_transform(img_path)
    pos_cls = get_transformed_positions_and_classes(img_path, board_pos=False)
    for d in pos_cls:
        x, y = d["position"]
        cv2.circle(img, (int(x), int(y)), 5, (255, 0, 0), -1)
    plt.imshow(img)
    plt.show()


# print(get_positions_and_classes("/content/cropped_images/frame_0001.jpg"))
# print(get_transformed_positions_and_classes("/content/cropped_images/frame_0001.jpg"))
# mark_red_dots("/content/chess-piece--2/train/images/00bc0cacffdebe6b11bdeec56f63ee49_jpg.rf.e7e6a567661410da0e2ff7b04d785f31.jpg")

# Extract img from videos

In [11]:


def extract_frames_by_fps(video_path, output_directory, frames_per_second, frame_prefix="frame", start_frame=0):
    """
    Extracts frames from a video at a specified frame rate (frames per second).

    Args:
        video_path (str): Path to the video file.
        output_directory (str): Directory to save the extracted frames.
        frames_per_second (int): Number of frames to extract per second.
        frame_prefix (str): Prefix for the frame filenames (default: "frame").
        start_frame (int): Starting frame number for naming (default: 0).

    Returns:
        int: Total number of frames extracted.
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return 0

    # Get video frame rate and total frames
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(video_fps // frames_per_second)  # Interval between frames to extract

    frame_count = start_frame
    frame_index = 0

    os.makedirs(output_directory, exist_ok=True)

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Save only frames at the specified interval
        if frame_index % frame_interval == 0:
            frame_filename = f"{output_directory}/{frame_prefix}_{frame_count:04d}.jpg"
            cv2.imwrite(frame_filename, frame)
            print(f"Saved: {frame_filename}")
            frame_count += 1

        frame_index += 1

    # Release the video capture object
    cap.release()
    cv2.destroyAllWindows()

    print(f"Total frames extracted: {frame_count - start_frame}")
    return frame_count - start_frame




In [12]:

def remove_black_bar(input_folder, output_folder, size=(640, 640)):
    """
    Processes all images in a folder by removing black borders and resizing them.

    Args:
        input_folder (str): Path to the folder containing input images.
        output_folder (str): Path to the folder to save processed images.
        size (tuple): Target size for the resized images (default is 640x640).

    Returns:
        None
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name)

        # Check if the file is an image (optional, based on file extension)
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            # Load the image
            image = cv2.imread(input_path)
            if image is None:
                print(f"Error: Could not load image {file_name}")
                continue

            # Convert to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Thresholding to create a binary mask
            _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

            # Find contours to detect the chessboard region
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contours:
                # Find the largest contour, assuming it's the chessboard
                largest_contour = max(contours, key=cv2.contourArea)

                # Get a bounding rectangle for the largest contour
                x, y, w, h = cv2.boundingRect(largest_contour)

                # Crop the image to the bounding rectangle
                cropped_image = image[y:y+h, x:x+w]

                # Resize the cropped image to the specified size
                resized_image = cv2.resize(cropped_image, size)

                # Save the processed image
                cv2.imwrite(output_path, resized_image)
                print(f"Processed image saved at: {output_path}")
            else:
                print(f"No contours found in image: {file_name}")
        else:
            print(f"Skipped non-image file: {file_name}")




In [13]:
def detect_hand(img_path, plot=False):
    """
    Detects whether a hand (based on skin tone) is present in the 640x640 image.

    Args:
        img_path (str): Path to the input image.

    Returns:
        bool: True if a hand (skin tone) is detected, False otherwise.
    """
    # Load the image
    image = perspective_transform(img_path)
    if plot:
        plt.imshow(image)
        plt.show()
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define HSV range for skin tones (adjust these values as needed)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)  # Lower HSV range
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)  # Upper HSV range

    # Create a binary mask for skin tones
    mask = cv2.inRange(hsv_image, lower_skin, upper_skin)

    # Apply morphological operations to reduce noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)  # Remove small noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  # Close small holes

    # Count non-zero pixels in the mask (skin-colored regions)
    skin_pixel_count = cv2.countNonZero(mask)
    # Set detection threshold based on 640x640 resolution
    detection_threshold = 30000
    is_hand_detected = skin_pixel_count > detection_threshold
    print(skin_pixel_count)
    # Debugging: Uncomment to visualize the mask
    # print("pixel count =", skin_pixel_count)
    # plt.imshow(mask)
    # plt.show()
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    return is_hand_detected

# detect_hand("/content/extracted_4_Move_studet/frame_0003.jpg", plot=True)

In [14]:
def create_img_frame_folder(video_path):
    file_name = video_path.split("/")[-1].split(".")[0]
    output_directory = "/content/extracted_" + file_name
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    extract_frames_by_fps(video_path, output_directory, 1)
    remove_black_bar(output_directory, output_directory)

    return output_directory

def get_board_pos(x, y):
    x_row = "hgfedcba"
    y_row = "12345678"
    return x_row[int(x)//80] + y_row[int(y)//80]


def convert_board_positions(img_path):
    pos_cls = get_transformed_positions_and_classes(img_path, board_pos=False)
    new_pos_cls = []
    for d in pos_cls:
        new_pos_cls.append({"position": get_board_pos(d["position"][0], d["position"][1]), "class_name": d["class_name"]})
    return new_pos_cls

def is_valid_img(img_path): #not blocked by hand or can't find 4 corners
    if detect_hand(img_path):
        print("detect hand")
        return False
    if len(remove_close_corners(get_corners(find_corners(img_path)))) != 4:
        print("len(corners) is not 4")
        return False
    return True






In [15]:


def generate_fen(img_path, active_color='w'):

    positions = get_transformed_positions_and_classes(img_path)
    # Create an empty board
    board = chess.Board.empty()

    # Mapping class names to chess symbols
    piece_map = {
        'white-king': 'K', 'black-king': 'k', 'white-queen': 'Q', 'black-queen': 'q', 'white-rook': 'R', 'black-rook': 'r', 'white-bishop': 'B', 'black-bishop': 'b', 'white-knight': 'N', 'black-knight': 'n', 'white-pawn': 'P', 'black-pawn': 'p'
    }
    # Place pieces on the board
    for item in positions:
        position = item['position']
        class_name = item['class_name']
        piece_symbol = piece_map.get(class_name)
        if piece_symbol:
            board.set_piece_at(chess.parse_square(position), chess.Piece.from_symbol(piece_symbol))

    # Set the active color
    board.turn = (active_color == 'w')

    # Return the FEN string
    return board.fen()




def display_board(img_path="", board=None):
    if img_path != "":
        fen = generate_fen(img_path)
        board = chess.Board(fen)
        svg = chess.svg.board(board=board, size=400, orientation=chess.BLACK)
        display(SVG(svg))
    else:
        svg = chess.svg.board(board=board, size=400, orientation=chess.BLACK)
        display(SVG(svg))


def get_board_diff(previous_frame, current_frame):
    """
    Identifies the differences between two frames by comparing positions and classes.

    Args:
        previous_frame: The previous frame data.
        current_frame: The current frame data.

    Returns:
        tuple: Two lists representing the differences in positions and classes:
               - prev_pos: Items present in previous_frame but not in current_frame.
               - current_pos: Items present in current_frame but not in previous_frame.
    """
    # Get transformed positions and classes for both frames
    prev_pos = get_transformed_positions_and_classes(previous_frame)
    current_pos = get_transformed_positions_and_classes(current_frame)

    # Convert dictionaries to tuples of sorted key-value pairs for comparison
    prev_set = {tuple(sorted(d.items())) for d in prev_pos}
    curr_set = {tuple(sorted(d.items())) for d in current_pos}

    # Find common items
    common_items = prev_set & curr_set

    # Filter out common items from the original lists
    prev_pos = [d for d in prev_pos if tuple(sorted(d.items())) not in common_items]
    current_pos = [d for d in current_pos if tuple(sorted(d.items())) not in common_items]

    return prev_pos, current_pos



f1 = "/content/extracted_4_Move_studet/frame_0002.jpg"
f2 = "/content/extracted_4_Move_studet/frame_0011.jpg"

print(get_board_diff(f1, f2))
display_board(img_path = f1)
display_board(img_path = f2)

FileNotFoundError: /content/extracted_4_Move_studet/frame_0002.jpg does not exist

In [ ]:
def generate_move(prev_pos, curr_pos):
    san_extended = {
    "black-king": "K",
    "black-queen": "Q",
    "black-rook": "R",
    "black-bishop": "B",
    "black-knight": "N",
    "black-pawn": "",
    "white-king": "K",
    "white-queen": "Q",
    "white-rook": "R",
    "white-bishop": "B",
    "white-knight": "N",
    "white-pawn": "",
    }
    if len(prev_pos) == 0:
        return None
    if len(prev_pos) == 1 and len(curr_pos) == 1:
        return san_extended[curr_pos[0]["class_name"]] + curr_pos[0]["position"]
    if len(prev_pos) == 2 and len(curr_pos) == 1:
        if curr_pos[0]["position"] == prev_pos[0]["position"]:
            return san_extended[curr_pos[0]["class_name"]] + curr_pos[0]["position"]
        else:
            return san_extended[prev_pos[1]["class_name"]] + prev_pos[1]["position"]

    print("prev_pos", prev_pos)
    print("curr_pos", curr_pos)
    if len(prev_pos) != 0 and len(curr_pos) != 0:
        return san_extended[curr_pos[0]["class_name"]] + curr_pos[0]["position"]
    else:
        return None

def create_PGN(video_path):
    folder_path = create_img_frame_folder(video_path)
    frames = [os.path.join(folder_path, f) for f in sorted(os.listdir(folder_path)) if f.lower().endswith('.jpg')]
    valid_frames = [frame for frame in frames if is_valid_img(frame)]

    first_frame = valid_frames[0]


    moves = []
    for i, frame in enumerate(valid_frames):
        if i == 0:
            pass
        elif i == 1:
            pos_before, pos_after = get_board_diff(first_frame, frame)
            move = generate_move(pos_before, pos_after)
            if move != None:
                moves.append(move)
        else:
            pos_before, pos_after = get_board_diff(valid_frames[i-1], frame)
            move = generate_move(pos_before, pos_after)
            if move != None:
                moves.append(move)

    paired_moves = [(moves[i], moves[i+1]) for i in range(0, len(moves) - 1, 2)]
    if len(moves) % 2 != 0:
        paired_moves.append((moves[-1], ""))
    string = ""
    for i in range(len(paired_moves)):
        string += str(i+1) + ". " + paired_moves[i][0] + " " + paired_moves[i][1] + " "

    return string
create_PGN("/content/4_Move_studet.mp4")



In [ ]:
def generate_pgn_from_video(video_path):
    """
    Generates a PGN notation file from image frames in a folder.

    Args:
        folder_path (str): Path to the folder containing image frames.
        detect_positions_func (function): Function to detect piece positions from an image frame.
            It should return a list of dictionaries with 'position' and 'class_name' keys.

    Returns:
        str: The PGN notation as a string.
    """
    # Get sorted list of image frames
    folder_path = create_img_frame_folder(video_path)
    frames = [os.path.join(folder_path, f) for f in sorted(os.listdir(folder_path)) if f.lower().endswith('.jpg')]

    # Filter out invalid frames
    valid_frames = [frame for frame in frames if is_valid_img(frame)]

    # Initialize variables
    pgn_game = chess.pgn.Game()
    node = pgn_game

    # Set initial FEN (assuming starting position unless provided)
    previous_fen = generate_fen(valid_frames[0], active_color="b")
    for frame in valid_frames:

        # Generate FEN from detected positions
        current_fen = generate_fen(frame)

        # Find move between previous and current FEN
        move = find_move(previous_fen, current_fen)

        # If no move is found, skip to the next frame
        if move is None:
            print(f"No valid move found between frames: {previous_fen} -> {current_fen}. Skipping frame.")
            continue
        else:
            print("found move")

        # Add move to PGN
        node = node.add_main_variation(chess.Move.from_uci(move))

        # Update previous FEN
        previous_fen = current_fen

    return str(pgn_game)


def remove_pgn_headers(pgn):

    # Use regex to remove lines starting with `[`
    clean_pgn = re.sub(r'^\[.*\]\n', '', pgn, flags=re.MULTILINE)
    return clean_pgn

